# Дектораторы

В этом домашнем задании мы напишем собственные дектораторы, которые будут менять системные объекты. Но для начала мы с вами познакомимся с функцией `write`.

In [1]:
import sys

sys.stdout.write('Hello, my friend!')

Hello, my friend!

17

Это метод объектов file-like классов, то есть классов, которые реализуют семантику "Меня можно создать, из меня можно прочитать и в меня можно записать".

Самый главный пример такого объекта -- объект `file`, являющийся результатом вызова фукнции `open()`. Для простоты и универсальности взаимодействия, стандартный ввод и стандартный вывод тоже являются файлами, из которых можно читать и в которые можно писать. 

In [2]:
output = open("./some_test_data.txt", 'w')

In [3]:
output.write('123')

3

In [4]:
output.close()

Как вы могли заметить, функция возвращает число записанных байт. Это важная часть контракта, которую нужно поддержать, если вы хотите как-то подменять эту функцию.

## Задача 1

Для начала, давайте подменим метод `write` у объекта `sys.stdin` на такую функцию, которая перед каждым вызовом оригинальной функции записи данных в `stdout` допечатывает к тексту текущую метку времени.

In [2]:
import datetime

original_write = sys.stdout.write

In [9]:
def my_write(string_text):
    if string_text == '\n':
        original_write(string_text)
        return
    timestamp = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    text_with_timestamp = f'[{timestamp}]: {string_text}'
    original_write(text_with_timestamp)

In [10]:
sys.stdout.write = my_write

In [11]:
print('1, 2, 3')

[2023-12-14 13:28:21]: 1, 2, 3


In [12]:
sys.stdout.write = original_write

Вывод должен был бы быть примерно таким:

```
[2021-12-05 12:00:00]: 1, 2, 3
```

## Задача 2

Упакуйте только что написанный код в декторатор. Весь вывод фукнции должен быть помечен временными метками так, как видно выше.

In [16]:
import datetime

def timed_output(function):
    def wrapper(*args, **kwargs):
        def my_write(string_text):
            if string_text == '\n':
                original_write(string_text)
                return
            timestamp = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            text_with_timestamp = f'[{timestamp}]: {string_text}'
            original_write(text_with_timestamp)
            
        original_write = sys.stdout.write
        sys.stdout.write = my_write
        function(*args, **kwargs)
        sys.stdout.write = original_write
    return wrapper

In [17]:
@timed_output
def print_greeting(name):
    print(f'Hello, {name}!')

In [18]:
print_greeting("Nikita")

[2023-12-14 13:30:21]: Hello, Nikita!


Вывод должен быть похож на следующий:

```
[2021-12-05 12:00:00]: Hello, Nikita!
```

## Задача 3

Напишите декторатор, который будет перенаправлять вывод фукнции в файл. 

Подсказка: вы можете заменить объект sys.stdout каким-нибудь другим объектом.

In [4]:
def redirect_output(filepath):
    def decorator(func):
        def wrapper(*args, **kwargs):
            original = sys.stdout
            with open(filepath, 'w') as output:
                sys.stdout = output
                res = func(*args, **kwargs)
                sys.stdout = original
            return res
        return wrapper
    return decorator

In [5]:
@redirect_output('function_output.txt')
def calculate():
    for power in range(1, 5):
        for num in range(1, 20):
            print(num ** power, end=' ')
        print()

In [6]:
calculate()

In [33]:
%cat function_output.txt

UsageError: Line magic function `%cat` not found.
